In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("proj2.ipynb")

# Project 2: Query Performance
## Due Date: Friday 10/07, 11:59 PM
## Assignment Details
In this project, we will explore how the database system optimizes query execution and how users can futher tune the performance of their queries.

This project works with the Lahman's Baseball Database, an open source collection of baseball statistics from 1871 to 2020. It contains a variety of data, like batting statistics, team stats, managerial records, Hall of Fame records, and much more.

**Note:** If at any point during the project, the internal state of the database or its tables have been modified in an undesirable way (i.e. a modification not resulting from the instructions of a question), restart your kernel and clear output and simply re-run the notebook as normal. This will shutdown you current connection to the database, which will prevent the issue of multiple connections to the database at any given point, and when re-running the notebook you will create a fresh database based on the provided Postgres dump.

## Logistics & Scoring Breakdown

For Data 101 students, this project is worth 15% of your grade. For Info 258 students, this project is worth 12% of your grade.

- Each coding question has **both public tests and hidden tests**. Roughly 50% of your coding grade will be made up of your score on the public tests released to you, while the remaining 50% will be made up of unreleased hidden tests.
- Public tests for multiple choice questions are for sanity check only (e.g. you are answering in the correct format). Partial credit will be awarded.
- Free-response questions will be manually graded. Please answer thoughtfully in complete sentences, drawing from knowledge in lectures and from your inspection of query plans.

This is an **individual project**. However, you’re welcome to collaborate with any other student in the class as long as it’s within the academic honesty guidelines.

Question | Points
--- | ---
0 | 1
1a	| 1
1bi	| 1
1bii	| 2
1c	| 1
1di	| 2
1dii	| 1
1ei	| 1
1eii	| 1
1eiii	| 1
1eiv	| 2
1ev	| 2
2a	| 1
2b	| 1
2c	| 2
2d	| 2
3a	| 1
3b	| 1
3c |	1
3d	| 3
4a	| 2
4b	| 1
4c	| 1
4di	| 2
4dii |	2
4ei	| 2
4eii | 	1
4eiii |	2
4eiv |	1
4ev |	1
4evi |	2
4evii |	2
5a | 2
5b | 2
5c | 1
5d | 2
6a | 2
6b | 1
6c | 1
6d | 2
6e | 2
7a | 1
7b | 1
7c | 2
8  | 6
**Total** | 72

In [2]:
# Run this cell to set up imports
import numpy as np
import pandas as pd

## Getting Connected
Similar to Project 1, we will be using the `ipython-sql` library to connect this notebook to a PostgreSQL database server on your JupyterHub account. Run the following cell to initiate the connection.

In [3]:
%reload_ext sql
%sql postgresql://jovyan@127.0.0.1:5432/postgres

## Setting up the Database
The following cell will create the `baseball` database (if needed), unzip the Postgres dump of the Lahman's Baseball Database, populate the `baseball` database with the desired tables and data, and finally display all databases associated with the Postgres instance. After running the cell, you should see the `baseball` database in the generated list of databases outputted by `%sql \l`.

**Note:** If you run into the **role does not exist** error, feel free to ignore it. It does not affect data import.

In [4]:
!psql -h localhost -c 'DROP DATABASE IF EXISTS baseball'
!psql -h localhost -c 'CREATE DATABASE baseball'
!gzip -dc data/lahman.pgdump.gz | psql -h localhost -d baseball -f -
!psql -h localhost -c 'SET max_parallel_workers_per_gather = 0;'
%sql \l

DROP DATABASE
CREATE DATABASE
SET
SET
SET
SET
 set_config 
------------
 
(1 row)

SET
SET
SET
CREATE EXTENSION
COMMENT
SET
CREATE TABLE
psql:<stdin>:49: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:80: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:96: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:112: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:129: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:146: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:178: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:210: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:223: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:251: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:267: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:295: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:322: ERROR:  

[('baseball', 'jovyan', 'UTF8', 'en_US.utf8', 'en_US.utf8', None),
 ('imdb', 'jovyan', 'UTF8', 'en_US.utf8', 'en_US.utf8', None),
 ('jovyan', 'jovyan', 'UTF8', 'en_US.utf8', 'en_US.utf8', None),
 ('postgres', 'jovyan', 'UTF8', 'en_US.utf8', 'en_US.utf8', None),
 ('template0',
  'jovyan',
  'UTF8',
  'en_US.utf8',
  'en_US.utf8',
  '=c/jovyan\njovyan=CTc/jovyan'),
 ('template1',
  'jovyan',
  'UTF8',
  'en_US.utf8',
  'en_US.utf8',
  '=c/jovyan\njovyan=CTc/jovyan')]

Now, run the following cell to connect to the `baseball` database. There should be no errors after running the following cell.

In [5]:
%sql postgresql://jovyan@127.0.0.1:5432/baseball

To ensure that the connection to the database has been established, let's try grabbing the first 5 rows from the `halloffame` table.

In [6]:
%%sql
SELECT * FROM halloffame LIMIT 5

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
5 rows affected.


[('cobbty01', 1936, 'BBWAA', 226, 170, 222, 'Y', 'Player', None),
 ('ruthba01', 1936, 'BBWAA', 226, 170, 215, 'Y', 'Player', None),
 ('wagneho01', 1936, 'BBWAA', 226, 170, 215, 'Y', 'Player', None),
 ('mathech01', 1936, 'BBWAA', 226, 170, 205, 'Y', 'Player', None),
 ('johnswa01', 1936, 'BBWAA', 226, 170, 189, 'Y', 'Player', None)]

Run the following cell for grading purposes.

In [7]:
!mkdir -p results

## Table Descriptions
In its entirety the Lahman's Baseball Database contains 27 tables containing a variety of statistics for players, teams, games, schools, etc. For simplicity, this project will focus on a subset of the tables:
* `appearances`: details on the positions each player appeared at
* `batting`: batting statistics for each player
* `collegeplaying`: list of players and the colleges they attended
* `halloffame`: Hall of Fame voting data
* `people`: player information (name, date of birth, and biographical info)
* `salaries`: player salary data
* `schools`: list of colleges that players attended

As a reminder from Project 1, `%sql \d <table_name>` is helpful for identifying the columns in a table.

## Question 0: PostgreSQL Explain Analyze
**Please read through this section carefully, as a vast majority of the project will require you to inspect query plans via interpreting the output of the explain analyze command.**

To inspect the query plan for a given query, create a variable storing the query as a string and invoke a `psql` shell command to `explain analyze` the query: 

`your_variable = "__REPLACE_ME_WITH_QUERY__"`

`!psql -h localhost -d baseball -c "explain analyze $your_variable"`

Take a look at the following sample query plan.

![title](data/sample_query.png)

It is highly recommended to read through [this article](https://www.cybertec-postgresql.com/en/how-to-interpret-postgresql-explain-analyze-output/) to see how you can interpret the output above. Everything before "Tools to interpret Explain Analyze output" is useful.

Here are some key things to note:
- There are two cost values: the first is the **startup cost** (cost to return the first row) and the second is the **total cost** (cost to return all rows).
- The unit for the estimated query cost is arbitrary (1 is the cost for reading an 8kB page during a sequential scan)
- When we ask you to identify the query cost, we are looking for the **total cost**
- When we ask you to identify the query time, we are looking for the **execution time** (in ms)
- Feel free to round the query cost / time to the nearest integer, but we'll accept anything more exact
- We recognzie that the execution time may vary between different cell executions, so the autograder will tolerate a reasonable range

Now, inspect the query plan above, and record the **cost** and **time** for the sample query.

<!--
BEGIN QUESTION
name: q0
points: 1
-->

In [8]:
sample_query_cost = 1194
sample_query_timing = 186

In [9]:
grader.check("q0")

q0 results: All test cases passed!

## Question 1: Queries and Views
### Question 1a:
Write a query that finds `namefirst`, `namelast`, `playerid` and `yearid` of all people who were successfully inducted into the Hall of Fame. **Note**: Your query should **NOT** use any sub-queries.

In [10]:
%%sql
SELECT * FROM PEOPLE LIMIT 5

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
5 rows affected.


[('aardsda01', 1981, 12, 27, 'USA', 'CO', 'Denver', None, None, None, None, None, None, 'David', 'Aardsma', 'David Allan', 215, 75.0, 'R', 'R', '2004-04-06', '2015-08-23', 'aardd001', 'aardsda01'),
 ('aaronha01', 1934, 2, 5, 'USA', 'AL', 'Mobile', None, None, None, None, None, None, 'Hank', 'Aaron', 'Henry Louis', 180, 72.0, 'R', 'R', '1954-04-13', '1976-10-03', 'aaroh101', 'aaronha01'),
 ('aaronto01', 1939, 8, 5, 'USA', 'AL', 'Mobile', 1984, 8, 16, 'USA', 'GA', 'Atlanta', 'Tommie', 'Aaron', 'Tommie Lee', 190, 75.0, 'R', 'R', '1962-04-10', '1971-09-26', 'aarot101', 'aaronto01'),
 ('aasedo01', 1954, 9, 8, 'USA', 'CA', 'Orange', None, None, None, None, None, None, 'Don', 'Aase', 'Donald William', 190, 75.0, 'R', 'R', '1977-07-26', '1990-10-03', 'aased001', 'aasedo01'),
 ('abadan01', 1972, 8, 25, 'USA', 'FL', 'Palm Beach', None, None, None, None, None, None, 'Andy', 'Abad', 'Fausto Andres', 184, 73.0, 'L', 'L', '2001-09-10', '2006-04-13', 'abada001', 'abadan01')]

In [11]:
%%sql result_1a <<
SELECT a.namefirst, a.namelast, b.playerid, b.yearid
FROM people as a
JOIN halloffame as b
ON a.playerid = b.playerid
WHERE inducted = 'Y';

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
323 rows affected.
Returning data to local variable result_1a


In [12]:
# Do not delete/edit this cell
result_1a.DataFrame().sort_values(['playerid', 'yearid']).to_csv('results/result_1a.csv', index=False)

In [13]:
result_1a

[('Ty', 'Cobb', 'cobbty01', 1936),
 ('Babe', 'Ruth', 'ruthba01', 1936),
 ('Honus', 'Wagner', 'wagneho01', 1936),
 ('Christy', 'Mathewson', 'mathech01', 1936),
 ('Walter', 'Johnson', 'johnswa01', 1936),
 ('Nap', 'Lajoie', 'lajoina01', 1937),
 ('Tris', 'Speaker', 'speaktr01', 1937),
 ('Cy', 'Young', 'youngcy01', 1937),
 ('Morgan', 'Bulkeley', 'bulkemo99', 1937),
 ('Ban', 'Johnson', 'johnsba99', 1937),
 ('Connie', 'Mack', 'mackco01', 1937),
 ('John', 'McGraw', 'mcgrajo01', 1937),
 ('George', 'Wright', 'wrighge01', 1937),
 ('Pete', 'Alexander', 'alexape01', 1938),
 ('Alexander', 'Cartwright', 'cartwal99', 1938),
 ('Henry', 'Chadwick', 'chadwhe99', 1938),
 ('George', 'Sisler', 'sislege01', 1939),
 ('Eddie', 'Collins', 'collied01', 1939),
 ('Willie', 'Keeler', 'keelewi01', 1939),
 ('Cap', 'Anson', 'ansonca01', 1939),
 ('Charlie', 'Comiskey', 'comisch01', 1939),
 ('Candy', 'Cummings', 'cummica01', 1939),
 ('Buck', 'Ewing', 'ewingbu01', 1939),
 ('Lou', 'Gehrig', 'gehrilo01', 1939),
 ('Old Hoss', 'Radbourn', 'radboch01', 1939),
 ('Al', 'Spalding', 'spaldal01', 1939),
 ('Rogers', 'Hornsby', 'hornsro01', 1942),
 ('Kenesaw', 'Landis', 'landike99', 1944),
 ('Roger', 'Bresnahan', 'bresnro01', 1945),
 ('Dan', 'Brouthers', 'broutda01', 1945),
 ('Fred', 'Clarke', 'clarkfr01', 1945),
 ('Jimmy', 'Collins', 'colliji01', 1945),
 ('Ed', 'Delahanty', 'delahed01', 1945),
 ('Hugh', 'Duffy', 'duffyhu01', 1945),
 ('Hughie', 'Jennings', 'jennihu01', 1945),
 ('King', 'Kelly', 'kellyki01', 1945),
 ('Jim', "O'Rourke", 'orourji01', 1945),
 ('Wilbert', 'Robinson', 'robinwi01', 1945),
 ('Jesse', 'Burkett', 'burkeje01', 1946),
 ('Frank', 'Chance', 'chancfr01', 1946),
 ('Jack', 'Chesbro', 'chesbja01', 1946),
 ('Johnny', 'Evers', 'eversjo01', 1946),
 ('Clark', 'Griffith', 'griffcl01', 1946),
 ('Tommy', 'McCarthy', 'mccarto01', 1946),
 ('Joe', 'McGinnity', 'mcginjo01', 1946),
 ('Eddie', 'Plank', 'planked01', 1946),
 ('Joe', 'Tinker', 'tinkejo01', 1946),
 ('Rube', 'Waddell', 'wadderu01', 1946),
 ('Ed', 'Walsh', 'walshed01', 1946),
 ('Carl', 'Hubbell', 'hubbeca01', 1947),
 ('Frankie', 'Frisch', 'friscfr01', 1947),
 ('Mickey', 'Cochrane', 'cochrmi01', 1947),
 ('Lefty', 'Grove', 'grovele01', 1947),
 ('Herb', 'Pennock', 'pennohe01', 1948),
 ('Pie', 'Traynor', 'traynpi01', 1948),
 ('Charlie', 'Gehringer', 'gehrich01', 1949),
 ('Mordecai', 'Brown', 'brownmo01', 1949),
 ('Kid', 'Nichols', 'nichoki01', 1949),
 ('Mel', 'Ott', 'ottme01', 1951),
 ('Jimmie', 'Foxx', 'foxxji01', 1951),
 ('Harry', 'Heilmann', 'heilmha01', 1952),
 ('Paul', 'Waner', 'wanerpa01', 1952),
 ('Dizzy', 'Dean', 'deandi01', 1953),
 ('Al', 'Simmons', 'simmoal01', 1953),
 ('Ed', 'Barrow', 'barroed99', 1953),
 ('Chief', 'Bender', 'bendech01', 1953),
 ('Tommy', 'Connolly', 'connoto99', 1953),
 ('Bill', 'Klem', 'klembi99', 1953),
 ('Bobby', 'Wallace', 'wallabo01', 1953),
 ('Harry', 'Wright', 'wrighha01', 1953),
 ('Rabbit', 'Maranville', 'maranra01', 1954),
 ('Bill', 'Dickey', 'dickebi01', 1954),
 ('Bill', 'Terry', 'terrybi01', 1954),
 ('Joe', 'DiMaggio', 'dimagjo01', 1955),
 ('Ted', 'Lyons', 'lyonste01', 1955),
 ('Dazzy', 'Vance', 'vanceda01', 1955),
 ('Gabby', 'Hartnett', 'hartnga01', 1955),
 ('Home Run', 'Baker', 'bakerfr01', 1955),
 ('Ray', 'Schalk', 'schalra01', 1955),
 ('Hank', 'Greenberg', 'greenha01', 1956),
 ('Joe', 'Cronin', 'cronijo01', 1956),
 ('Sam', 'Crawford', 'crawfsa01', 1957),
 ('Joe', 'McCarthy', 'mccarjo99', 1957),
 ('Zack', 'Wheat', 'wheatza01', 1959),
 ('Max', 'Carey', 'careyma01', 1961),
 ('Billy', 'Hamilton', 'hamilbi01', 1961),
 ('Bob', 'Feller', 'fellebo01', 1962),
 ('Jackie', 'Robinson', 'robinja02', 1962),
 ('Bill', 'McKechnie', 'mckecbi01', 1962),
 ('Edd', 'Roush', 'roushed01', 1962),
 ('John', 'Clarkson', 'clarkjo01', 1963),
 ('Elmer', 'Flick', 'flickel01', 1963),
 ('Sam', 'Rice', 'ricesa01', 1963),
 ('Eppa', 'Rixey', 'rixeyep01', 1963),
 ('Luke', 'Appling', 'applilu01', 1964),
 ('Red', 'Faber', 'faberre01', 1964),
 ('Burleigh', 'Grimes', 'grimebu01', 1964),
 ('Miller', 'Huggins', 

In [14]:
grader.check("q1a")

q1a results: All test cases passed!

### Question 1b:
In this question, we will compare the query you wrote in `Question 1a` against the provided query in `Question 1bi` by inspecting the query plans.

#### Question 1bi: 
Inspect the query plan for `provided_query` and the query you wrote in `Question 1a`. Record the **execution time** and **cost** for each query.

In [15]:
%%sql provided_query <<
SELECT namefirst, namelast, p.playerid, yearid
FROM people AS p, (SELECT * FROM halloffame WHERE inducted = 'Y') AS hof 
WHERE p.playerid = hof.playerid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
323 rows affected.
Returning data to local variable provided_query


In [16]:
provided_query_str = "SELECT namefirst, namelast, p.playerid, yearid FROM people AS p, (SELECT * FROM halloffame WHERE inducted = 'Y') AS hof WHERE hof.inducted = 'Y' AND p.playerid = hof.playerid;"
!psql -h localhost -d baseball -c "explain analyze $provided_query_str"

                                                           QUERY PLAN                                                           
--------------------------------------------------------------------------------------------------------------------------------
 Nested Loop  (cost=0.29..262.79 rows=21 width=278) (actual time=0.055..2.291 rows=323 loops=1)
   ->  Seq Scan on halloffame  (cost=0.00..96.39 rows=21 width=42) (actual time=0.026..0.591 rows=323 loops=1)
         Filter: ((inducted)::text = 'Y'::text)
         Rows Removed by Filter: 3868
   ->  Index Scan using master_pkey on people p  (cost=0.29..7.92 rows=1 width=274) (actual time=0.005..0.005 rows=1 loops=323)
         Index Cond: ((playerid)::text = (halloffame.playerid)::text)
 Planning Time: 0.627 ms
 Execution Time: 2.358 ms
(8 rows)



In [17]:
inducted_hof_query_str = "SELECT a.namefirst, a.namelast, b.playerid, b.yearid FROM people as a JOIN halloffame as b ON a.playerid = b.playerid WHERE inducted = 'Y'"
!psql -h localhost -d baseball -c "explain analyze $inducted_hof_query_str"

                                                           QUERY PLAN                                                           
--------------------------------------------------------------------------------------------------------------------------------
 Nested Loop  (cost=0.29..262.79 rows=21 width=278) (actual time=0.082..4.118 rows=323 loops=1)
   ->  Seq Scan on halloffame b  (cost=0.00..96.39 rows=21 width=42) (actual time=0.044..1.187 rows=323 loops=1)
         Filter: ((inducted)::text = 'Y'::text)
         Rows Removed by Filter: 3868
   ->  Index Scan using master_pkey on people a  (cost=0.29..7.92 rows=1 width=274) (actual time=0.008..0.008 rows=1 loops=323)
         Index Cond: ((playerid)::text = (b.playerid)::text)
 Planning Time: 0.958 ms
 Execution Time: 4.233 ms
(8 rows)



In [18]:
provided_query_cost = 959
provided_query_timing = 10.739
your_query_cost = 959
your_query_timing = 10.442

In [19]:
grader.check("q1bi")

q1bi results: All test cases passed!

#### Question 1bii:
Given your findings from inspecting the query plans of the two queries, answer the following question. Assign the variable `q1b_part2` to a list of answer choices which are true statements.

1. Consider the following statements:
    1. Both the queries have the same cost.
    1. The provided query has a faster execution time because it makes use of a subquery.
    1. The query you wrote has a faster execution time because it does not make use a subquery.
    1. The provided query has less cost because it makes use of a subquery.
    1. The query you wrote has less cost because it does not make use a subquery.
    1. The queries have the same output.
    1. The queries do not have the same output.
    
**Note:** Your answer should look like `q1b_part2 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q1bii
points: 2
-->

In [20]:
q1b_part2 =  ['A', 'F']

In [21]:
grader.check("q1bii")

q1bii results: All test cases passed!

### Question 1c:
Write a query that creates a view named `inducted_hof_ca` of the people who were successfully inducted into the Hall of Fame and played in college at a school located in California. For each player, return their `namefirst`, `namelast`, `playerid`, `schoolid`, and `yearid` ordered by the `yearid` and then the `playerid`. 

**Note**: For this query, `yearid` refers to player's year of induction into the Hall of Fame.

In [22]:
%%sql
SELECT * FROM collegeplaying LIMIT 5

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
5 rows affected.


[('aardsda01', 'pennst', 2001),
 ('aardsda01', 'rice', 2002),
 ('aardsda01', 'rice', 2003),
 ('abadan01', 'gamiddl', 1992),
 ('abadan01', 'gamiddl', 1993)]

In [23]:
%%sql result_1c <<
DROP VIEW IF EXISTS inducted_hof_ca;
CREATE VIEW inducted_hof_ca AS

SELECT join2table.namefirst,join2table.namelast, join2table.playerid, join2table.schoolid, join2table.yearid
FROM
(SELECT jointable.namefirst,jointable.namelast, jointable.playerid, jointable.yearid, c.schoolid
FROM (SELECT a.namefirst, a.namelast, b.playerid, b.yearid
FROM people as a
JOIN halloffame as b
ON a.playerid = b.playerid
WHERE inducted = 'Y') as jointable
JOIN collegeplaying as c
ON jointable.playerid = c.playerid) as join2table

JOIN schools as d

ON join2table.schoolid = d.schoolid

where d.schoolstate = 'CA'

ORDER BY join2table.yearid, join2table.playerid

;
SELECT * FROM inducted_hof_ca;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Done.
23 rows affected.
Returning data to local variable result_1c


In [24]:
# Do not delete/edit this cell
result_1c.DataFrame().sort_values(['playerid', 'schoolid', 'yearid']).to_csv('results/result_1c.csv', index=False)

In [25]:
result_1c

[('Jackie', 'Robinson', 'robinja02', 'ucla', 1962),
 ('Harry', 'Hooper', 'hoopeha01', 'stmarysca', 1971),
 ('Joe', 'Morgan', 'morgajo02', 'camerri', 1990),
 ('Tom', 'Seaver', 'seaveto01', 'cafrecc', 1992),
 ('Tom', 'Seaver', 'seaveto01', 'usc', 1992),
 ('Ozzie', 'Smith', 'smithoz01', 'calpoly', 2002),
 ('Ozzie', 'Smith', 'smithoz01', 'calpoly', 2002),
 ('Ozzie', 'Smith', 'smithoz01', 'calpoly', 2002),
 ('Ozzie', 'Smith', 'smithoz01', 'calpoly', 2002),
 ('Tony', 'Gwynn', 'gwynnto01', 'sandiegost', 2007),
 ('Tony', 'Gwynn', 'gwynnto01', 'sandiegost', 2007),
 ('Tony', 'Gwynn', 'gwynnto01', 'sandiegost', 2007),
 ('Dick', 'Williams', 'willidi02', 'capasad', 2008),
 ('Doug', 'Harvey', 'harvedo99', 'sandiegost', 2010),
 ('Doug', 'Harvey', 'harvedo99', 'sandiegost', 2010),
 ('Pat', 'Gillick', 'gillipa99', 'calavco', 2011),
 ('Pat', 'Gillick', 'gillipa99', 'usc', 2011),
 ('Pat', 'Gillick', 'gillipa99', 'usc', 2011),
 ('Randy', 'Johnson', 'johnsra05', 'usc', 2015),
 ('Randy', 'Johnson', 'johnsra05', 'usc', 2015),
 ('Randy', 'Johnson', 'johnsra05', 'usc', 2015),
 ('Trevor', 'Hoffman', 'hoffmtr01', 'cacypre', 2018),
 ('Trevor', 'Hoffman', 'hoffmtr01', 'cacypre', 2018)]

In [26]:
grader.check("q1c")

q1c results: All test cases passed!

### Question 1d:
For this question, we want to compute the count of players who were inducted into the Hall of Fame and played baseball at a college in California for each `schoolid` and `yearid` combination ordered by ascending `yearid`.

#### Question 1di:
Write two queries that accomplish this task -- one query using the view you created in the `Question 1c` and one query that does not use the view, common table expressions (CTEs), or any sub-queries.

In [27]:
%%sql
SELECT schoolid,yearid, COUNT(*) FROM inducted_hof_ca

GROUP BY schoolid, yearid order by yearid asc;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
13 rows affected.


[('ucla', 1962, 1),
 ('stmarysca', 1971, 1),
 ('camerri', 1990, 1),
 ('cafrecc', 1992, 1),
 ('usc', 1992, 1),
 ('calpoly', 2002, 4),
 ('sandiegost', 2007, 3),
 ('capasad', 2008, 1),
 ('sandiegost', 2010, 2),
 ('calavco', 2011, 1),
 ('usc', 2011, 2),
 ('usc', 2015, 3),
 ('cacypre', 2018, 2)]

In [28]:
%%sql with_view <<
SELECT schoolid,yearid, COUNT(*) FROM inducted_hof_ca

GROUP BY schoolid, yearid order by yearid asc;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
13 rows affected.
Returning data to local variable with_view


In [29]:
%%sql without_view <<
SELECT a.schoolid, c.yearid, COUNT(*) 
FROM collegeplaying AS a, schools AS b, halloffame AS c
WHERE c.playerid = a.playerid AND a.schoolid = b.schoolid AND b.schoolstate = 'CA' AND c.inducted = 'Y'

GROUP BY a.schoolid, c.yearid order by yearid asc;


 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
13 rows affected.
Returning data to local variable without_view


In [30]:
# Do not delete/edit this cell
with_view.DataFrame().sort_values(['schoolid', 'yearid']).to_csv('results/result_1di_view.csv', index=False)
without_view.DataFrame().sort_values(['schoolid', 'yearid']).to_csv('results/result_1di_no_view.csv', index=False)

In [31]:
grader.check("q1di")

q1di results: All test cases passed!

#### Question 1dii:
Fill in your queries from `Question 1di` and inspect the query plans for the two queries. Record the execution time and cost for each query.

In [32]:
with_view_query_str = "SELECT schoolid,yearid, COUNT(*) FROM inducted_hof_ca GROUP BY schoolid, yearid order by yearid asc"
!psql -h localhost -d baseball -c "explain analyze $with_view_query_str"

                                                                                    QUERY PLAN                                                                                     
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=325.73..325.75 rows=1 width=60) (actual time=4.462..4.472 rows=13 loops=1)
   Group Key: inducted_hof_ca.yearid, inducted_hof_ca.schoolid
   ->  Sort  (cost=325.73..325.74 rows=1 width=52) (actual time=4.457..4.460 rows=23 loops=1)
         Sort Key: inducted_hof_ca.yearid, inducted_hof_ca.schoolid
         Sort Method: quicksort  Memory: 26kB
         ->  Subquery Scan on inducted_hof_ca  (cost=325.71..325.72 rows=1 width=52) (actual time=4.410..4.415 rows=23 loops=1)
               ->  Sort  (cost=325.71..325.71 rows=1 width=326) (actual time=4.408..4.410 rows=23 loops=1)
                     Sort Key: b.yea

In [33]:
without_view_query_str = "SELECT a.schoolid, c.yearid, COUNT(*) FROM collegeplaying AS a, schools AS b, halloffame AS c WHERE c.playerid = a.playerid AND a.schoolid = b.schoolid AND b.schoolstate = 'CA' AND c.inducted = 'Y' GROUP BY a.schoolid, c.yearid order by yearid asc"
!psql -h localhost -d baseball -c"explain analyze $without_view_query_str"

                                                                     QUERY PLAN                                                                     
----------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=423.13..423.31 rows=9 width=60) (actual time=12.213..12.224 rows=13 loops=1)
   Group Key: c.yearid, a.schoolid
   ->  Sort  (cost=423.13..423.15 rows=9 width=52) (actual time=12.205..12.209 rows=23 loops=1)
         Sort Key: c.yearid, a.schoolid
         Sort Method: quicksort  Memory: 26kB
         ->  Nested Loop  (cost=28.44..422.99 rows=9 width=52) (actual time=4.229..12.156 rows=23 loops=1)
               ->  Hash Join  (cost=28.16..360.36 rows=86 width=86) (actual time=0.241..4.394 rows=2948 loops=1)
                     Hash Cond: ((a.schoolid)::text = (b.schoolid)::text)
                     ->  Seq Scan on collegeplaying a  (cost=0.00..286.50 rows=17350 width=

In [34]:
with_view_cost =520.69
with_view_timing = 16.879
without_view_cost =490.87
without_view_timing =12.244

In [35]:
grader.check("q1dii")

q1dii results: All test cases passed!

### Question 1e:
#### Question 1ei:
Now, let's try creating a materialized view named `inducted_hof_ca_mat` instead of the regular view from `Question 1c`.

In [36]:
%%sql inducted_hof_ca_materialized <<
DROP MATERIALIZED VIEW IF EXISTS inducted_hof_ca_mat;
CREATE MATERIALIZED VIEW inducted_hof_ca_mat AS

SELECT join2table.namefirst,join2table.namelast, join2table.playerid, join2table.schoolid, join2table.yearid
FROM
(SELECT jointable.namefirst,jointable.namelast, jointable.playerid, jointable.yearid, c.schoolid
FROM (SELECT a.namefirst, a.namelast, b.playerid, b.yearid
FROM people as a
JOIN halloffame as b
ON a.playerid = b.playerid
WHERE inducted = 'Y') as jointable
JOIN collegeplaying as c
ON jointable.playerid = c.playerid) as join2table

JOIN schools as d

ON join2table.schoolid = d.schoolid

where d.schoolstate = 'CA'

ORDER BY join2table.yearid, join2table.playerid
;
SELECT * FROM inducted_hof_ca_mat;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
23 rows affected.
23 rows affected.
Returning data to local variable inducted_hof_ca_materialized


In [37]:
# Do not delete/edit this cell
inducted_hof_ca_materialized.DataFrame().sort_values(['playerid', 'schoolid', 'yearid']).to_csv('results/result_1ei.csv', index=False)

In [38]:
grader.check("q1ei")

q1ei results: All test cases passed!

#### Question 1eii:

Now, rewrite the query from `Question 1d` to use the materialized view `inducted_hof_ca_mat`.

In [39]:
%%sql with_materialized_view <<
SELECT schoolid,yearid, COUNT(*) FROM inducted_hof_ca_mat

GROUP BY schoolid, yearid order by yearid asc;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
13 rows affected.
Returning data to local variable with_materialized_view


In [40]:
# Do not delete/edit this cell
with_materialized_view.DataFrame().sort_values(['schoolid', 'yearid']).to_csv('results/result_1eii.csv', index=False)

In [41]:
grader.check("q1eii")

q1eii results: All test cases passed!

#### Question 1eiii:
Inspect the query plan and record the execution time and cost of the query that uses the materialized view.

In [42]:
with_materialized_view_query_str = "SELECT schoolid,yearid, COUNT(*) FROM inducted_hof_ca_mat GROUP BY schoolid, yearid order by yearid asc"
!psql -h localhost -d baseball -c "explain analyze $with_materialized_view_query_str"

                                                          QUERY PLAN                                                          
------------------------------------------------------------------------------------------------------------------------------
 Sort  (cost=23.67..24.17 rows=200 width=60) (actual time=0.059..0.060 rows=13 loops=1)
   Sort Key: yearid
   Sort Method: quicksort  Memory: 25kB
   ->  HashAggregate  (cost=14.03..16.02 rows=200 width=60) (actual time=0.026..0.030 rows=13 loops=1)
         Group Key: yearid, schoolid
         Batches: 1  Memory Usage: 40kB
         ->  Seq Scan on inducted_hof_ca_mat  (cost=0.00..12.30 rows=230 width=52) (actual time=0.008..0.010 rows=23 loops=1)
 Planning Time: 0.299 ms
 Execution Time: 0.173 ms
(9 rows)



In [43]:
with_materialized_view_cost = 24.17
with_materialized_view_timing =0.265

In [44]:
grader.check("q1eiii")

q1eiii results: All test cases passed!

#### Question 1eiv:
Given your findings from inspecting the query plans, as well as your understanding of views and materialized views from lectures, answer the following question. Assign the variable `q1e_part4` to a list of all statements which are true.

1. Consider the following statements:
    1. Views will reduce the execution time and the cost of a query.
    1. Views will reduce the execution time of a query, but not the cost.
    1. Views will reduce the cost of a query, but not the execution time.
    1. Materialized views reduce the execution time and the cost of a query.
    1. Materialized views reduce the execution time, but not cost of a query.
    1. Materialized views reduce the cost of a query, but not the execution time.
    1. Materialized views will result in the same query plan as a query using views.
    1. Materialized views and views take the same time to create.
    1. Materialized views take less time to create than a view.
    1. Materialized views take more time to create than a view.
    
*Note:* Your answer should look like `q1e_part4 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q1eiv
points: 2
-->

In [45]:
q1e_part4 = ['D',"I"]

In [46]:
grader.check("q1eiv")

q1eiv results: All test cases passed!

<!-- BEGIN QUESTION -->

#### Question 1ev:

Explain your answer to the previous part based on your knowledge from lectures, and details from the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q1ev
manual: true
points: 2
-->

materialized views are stored as "tables" in the system.
So it is faster and cost less when trying to use materialized wiew because there is no need to re-compute.
However, there is a higher matainance cost to it.
In contarst, Views are computed each time that you need to use the view which cost more.
SO thats why D I are correct.

<!-- END QUESTION -->



## Question 2: Predicate Pushdown
In this question, we will explore the impact of predicates on a query's execution, particularly inspecting when the optimizer applies predicates.

### Question 2a:
Recall the `inducted_hof_ca` created in `Question 1c`. Inspect the query plan for a query that that gets all rows from the view, and record the execution time and cost.

In [47]:
query_view_str = "SELECT join2table.namefirst,join2table.namelast, join2table.playerid, join2table.schoolid, join2table.yearid FROM (SELECT jointable.namefirst,jointable.namelast, jointable.playerid, jointable.yearid, c.schoolid FROM (SELECT a.namefirst, a.namelast, b.playerid, b.yearid FROM people as a JOIN halloffame as b ON a.playerid = b.playerid WHERE inducted = 'Y') as jointable JOIN collegeplaying as c ON jointable.playerid = c.playerid) as join2table JOIN schools as d ON join2table.schoolid = d.schoolid where d.schoolstate = 'CA' ORDER BY join2table.yearid, join2table.playerid"
!psql -h localhost -d baseball -c "explain analyze $query_view_str"

                                                                           QUERY PLAN                                                                            
-----------------------------------------------------------------------------------------------------------------------------------------------------------------
 Sort  (cost=325.71..325.71 rows=1 width=326) (actual time=4.742..4.745 rows=23 loops=1)
   Sort Key: b.yearid, b.playerid
   Sort Method: quicksort  Memory: 26kB
   ->  Nested Loop  (cost=0.85..325.70 rows=1 width=326) (actual time=1.836..4.685 rows=23 loops=1)
         ->  Nested Loop  (cost=0.57..319.95 rows=19 width=326) (actual time=0.203..4.217 rows=137 loops=1)
               ->  Nested Loop  (cost=0.29..262.79 rows=21 width=316) (actual time=0.050..2.608 rows=323 loops=1)
                     ->  Seq Scan on halloffame b  (cost=0.00..96.39 rows=21 width=42) (actual time=0.026..0.671 rows=323 loops=1)
                           Filter: ((inducted)::text = 'Y'::

In [48]:
query_view_cost = 528.30
query_view_timing = 6.303

In [49]:
grader.check("q2a")

q2a results: All test cases passed!

### Question 2b:
Now, add a filter to only return rows from `inducted_hof_ca` where the year is later than 2010. Inspect the query plan and record the execution time and cost.

In [50]:
%%sql result_2b <<
SELECT * FROM inducted_hof_ca where yearid >2010;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
8 rows affected.
Returning data to local variable result_2b


In [51]:
query_view_with_filter_str = "SELECT * FROM inducted_hof_ca where yearid >2010"
!psql -h localhost -d baseball -c "explain analyze $query_view_with_filter_str"

                                                                           QUERY PLAN                                                                           
----------------------------------------------------------------------------------------------------------------------------------------------------------------
 Sort  (cost=185.88..185.89 rows=1 width=326) (actual time=2.218..2.221 rows=8 loops=1)
   Sort Key: b.yearid, b.playerid
   Sort Method: quicksort  Memory: 25kB
   ->  Nested Loop  (cost=0.85..185.87 rows=1 width=326) (actual time=1.186..2.096 rows=8 loops=1)
         ->  Nested Loop  (cost=0.57..184.05 rows=6 width=326) (actual time=1.100..1.861 rows=27 loops=1)
               ->  Nested Loop  (cost=0.29..165.00 rows=7 width=316) (actual time=0.968..1.461 rows=31 loops=1)
                     ->  Seq Scan on halloffame b  (cost=0.00..106.86 rows=7 width=42) (actual time=0.918..0.995 rows=31 loops=1)
                           Filter: ((yearid > 2010) AND ((inducted)::

In [52]:
query_view_with_filter_cost = 209.85
query_view_with_filter_timing =  1.091

In [53]:
grader.check("q2b")

q2b results: All test cases passed!

### Question 2c:
Given your findings from inspecting the query plans of queries from `Question 2a-b`, answer the following question. Assign the variable `q2c` to a list of all statements which are true.

1. Consider the following statements:
    1. Adding a filter lowered the cost.
    1. Adding a filter increased the cost.
    1. Adding a filter did not change the cost.
    1. Adding a filter increased the execution time.
    1. Adding a filter decreased the execution time.
    1. Adding a filter did not change the execution time.
    1. No statement is true.
    
    
**Note:** Your answer should look like `q2c = ['A', 'B']`

<!--
BEGIN QUESTION
name: q2c
points: 2
-->

In [54]:
q2c = ["A","E"]

In [55]:
grader.check("q2c")

q2c results: All test cases passed!

<!-- BEGIN QUESTION -->

#### Question 2d:

Explain your answer based on your knowledge from lectures, and details from the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q2d
manual: true
points: 2
-->

Filtering can lower the cost becuase it removes rows and reduce the size of the data. Thus when we do other table operations such as join and group the cost and time is lower. That's why I choode A E.

<!-- END QUESTION -->



## Question 3: Join Approaches

In this question, we'll explore different join approaches (Nested Loop Join, Merge Join, Hash Join) and discuss how the query optimizer picks the best approach.

### Question 3a:
Perform an inner join on the `people` and `collegeplaying` tables on the `playerid` column. Project all columns.

In [56]:
%%sql q3a_join <<
SELECT * FROM people JOIN collegeplaying ON people.playerid = collegeplaying.playerid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
17350 rows affected.
Returning data to local variable q3a_join


Inspect the query plan for the query you wrote above.

In [57]:
q3a_query_str = "SELECT * FROM people JOIN collegeplaying ON people.playerid = collegeplaying.playerid"
!psql -h localhost -d baseball -c "explain analyze $q3a_query_str"

                                                          QUERY PLAN                                                           
-------------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=503.38..1562.91 rows=17350 width=1719) (actual time=7.061..22.906 rows=17350 loops=1)
   Hash Cond: ((people.playerid)::text = (collegeplaying.playerid)::text)
   ->  Seq Scan on people  (cost=0.00..619.70 rows=19370 width=1698) (actual time=0.038..3.862 rows=19370 loops=1)
   ->  Hash  (cost=286.50..286.50 rows=17350 width=21) (actual time=6.746..6.748 rows=17350 loops=1)
         Buckets: 32768  Batches: 1  Memory Usage: 1203kB
         ->  Seq Scan on collegeplaying  (cost=0.00..286.50 rows=17350 width=21) (actual time=0.022..1.977 rows=17350 loops=1)
 Planning Time: 1.265 ms
 Execution Time: 23.891 ms
(8 rows)



Given your findings from inspecting the query plan, answer the following question. Assign the variable `q3a` to a list of all the options which are true for the proposed statement.

1. Which join approach did the query optimizer choose?
    1. Nested Loop Join
    1. Merge Join
    1. Hash Join
    1. None of the Above

**Note:** Your answer should be formatted as follows: `q3a = ['A', 'B']`

<!--
BEGIN QUESTION
name: q3a
points: 1
-->

In [58]:
q3a = ["C"]

In [59]:
grader.check("q3a")

q3a results: All test cases passed!

### Question 3b

Similar to Question 3a, perform an inner join on the `people` and `collegeplaying` tables on the `playerid` column. Project all columns.

In addition, **sort your output by `playerid`.**

In [60]:
%%sql q3b_join <<
SELECT * FROM people JOIN collegeplaying ON people.playerid = collegeplaying.playerid order by collegeplaying.playerid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
17350 rows affected.
Returning data to local variable q3b_join


Inspect the query plan for the query you wrote above.

In [61]:
q3b_query_str = "SELECT * FROM people JOIN collegeplaying ON people.playerid = collegeplaying.playerid order by collegeplaying.playerid"
!psql -h localhost -d baseball -c "explain analyze $q3b_query_str"

                                                                        QUERY PLAN                                                                        
----------------------------------------------------------------------------------------------------------------------------------------------------------
 Merge Join  (cost=0.57..3184.67 rows=17350 width=1719) (actual time=0.099..24.890 rows=17350 loops=1)
   Merge Cond: ((people.playerid)::text = (collegeplaying.playerid)::text)
   ->  Index Scan using master_pkey on people  (cost=0.29..2298.84 rows=19370 width=1698) (actual time=0.034..5.993 rows=19368 loops=1)
   ->  Index Only Scan using collegeplaying_pkey on collegeplaying  (cost=0.29..620.54 rows=17350 width=21) (actual time=0.059..3.274 rows=17350 loops=1)
         Heap Fetches: 0
 Planning Time: 1.330 ms
 Execution Time: 25.816 ms
(7 rows)



Given your findings from inspecting the query plan, answer the following question. Assign the variable `q3b` to a list of all the options which are true for the proposed statement.

1. Which join approach did the query optimizer choose?
    1. Nested Loop Join
    1. Merge Join
    1. Hash Join
    1. None of the Above

**Note:** Your answer should be formatted as follows: `q3b = ['A', 'B']`

<!--
BEGIN QUESTION
name: q3b
points: 1
-->

In [62]:
q3b = ["B"]

In [63]:
grader.check("q3b")

q3b results: All test cases passed!

### Question 3c
Write a query to retrieve all possible player pair combinations. Project all columns, but **limit to 1000 rows** to ensure your query doesn't take an exorbitant amount of time to run.

**Hint:** You can do this by performing an inner join of the `people` table on itself with an inequality condition.

In [64]:
%%sql q3c_join <<
SELECT * FROM people AS a JOIN people AS b ON a.playerid > b.playerid  LIMIT 1000;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
1000 rows affected.
Returning data to local variable q3c_join


Inspect the query plan for the query you wrote above.

In [65]:
q3c_query_str = "SELECT * FROM people AS a JOIN people AS b ON a.playerid > b.playerid  LIMIT 1000"
!psql -h localhost -d baseball -c "explain analyze $q3c_query_str"

                                                                QUERY PLAN                                                                 
-------------------------------------------------------------------------------------------------------------------------------------------
 Limit  (cost=0.29..27.85 rows=1000 width=292) (actual time=0.040..0.736 rows=1000 loops=1)
   ->  Nested Loop  (cost=0.29..3447679.05 rows=125065633 width=292) (actual time=0.039..0.660 rows=1000 loops=1)
         ->  Seq Scan on people a  (cost=0.00..619.70 rows=19370 width=146) (actual time=0.015..0.020 rows=46 loops=1)
         ->  Index Scan using master_pkey on people b  (cost=0.29..113.39 rows=6457 width=146) (actual time=0.004..0.006 rows=22 loops=46)
               Index Cond: ((playerid)::text < (a.playerid)::text)
 Planning Time: 0.855 ms
 Execution Time: 0.845 ms
(7 rows)



Given your findings from inspecting the query plan, answer the following question. Assign the variable `q3c` to a list of all the options which are true for the proposed statement.

1. Which join approach did the query optimizer choose?
    1. Nested Loop Join
    1. Merge Join
    1. Hash Join
    1. None of the Above

**Note:** Your answer should be formatted as follows: `q3c = ['A', 'B']`

<!--
BEGIN QUESTION
name: q3c
points: 1
-->

In [66]:
q3c = ["A"]

In [67]:
grader.check("q3c")

q3c results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 3d
Please discuss your findings in the previous parts. In particular, we are interested in hearing why you think the query optimizer chooses the ultimate join approach in each of the above three scenarios. Feel free to discuss the pros and cons of each join approach as well.

If you feel stuck, here are some things to consider: Does a non-equijoin constrain us to certain join approaches? What's an added benefit in regards to the output of merge join?

<!--
BEGIN QUESTION
name: q3d
manual: true
points: 3
-->

When the table is not sorted, the system perform Hash Join, which hash R and S into 2 different buckets and
read all tuples hashed to the bucket from both R and S at a time and perform join. It is fast because sequence doesn't matter for this approach.

When the table is sorted, the system perform Merge Join, which sorted two tables first and then merge and match tuples across runs by walking down the runs in sorted order. The euqaljoin is faster but the table must be in sorted order in the first place in order to reduce cost.

When trying to find every copmbination of tuples pair, the system uses nested loop join that Match tuples across pairs of differente blocks. Since it scan through each block.

<!-- END QUESTION -->



## Question 4: Indexes

### Question 4a:
Write a query that outputs the `playerid` and average `salary` for each player that only batted in 10 games (the number of games in which a player batted can be found in the `g_batting` column of the `appearances` table). Your query should join the `salaries` and `appearances` table on all the common columns `yearid`, `teamid`, and `playerid`, so feel free to use a natural join.

In [68]:
%%sql
SELECT * FROM salaries LIMIT 5

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
5 rows affected.


[(1985, 'ATL', 'NL', 'barkele01', 870000.0),
 (1985, 'ATL', 'NL', 'bedrost01', 550000.0),
 (1985, 'ATL', 'NL', 'benedbr01', 545000.0),
 (1985, 'ATL', 'NL', 'campri01', 633333.0),
 (1985, 'ATL', 'NL', 'ceronri01', 625000.0)]

In [69]:
%%sql
SELECT * FROM appearances LIMIT 5

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
5 rows affected.


[(1871, 'TRO', 'NA', 'abercda01', 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0),
 (1871, 'RC1', 'NA', 'addybo01', 25, 25, 25, 25, 0, 0, 0, 22, 0, 3, 0, 0, 0, 0, 0, 0, 0),
 (1871, 'CL1', 'NA', 'allisar01', 29, 29, 29, 29, 0, 0, 0, 2, 0, 0, 0, 29, 0, 29, 0, 0, 0),
 (1871, 'WS3', 'NA', 'allisdo01', 27, 27, 27, 27, 0, 27, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 (1871, 'RC1', 'NA', 'ansonca01', 25, 25, 25, 25, 0, 5, 1, 2, 20, 0, 1, 0, 0, 1, 0, 0, 0)]

In [70]:
%%sql
SELECT a.playerid, a.salary FROM salaries AS a NATURAL JOIN appearances AS b WHERE 
b.g_batting = 10 LIMIT 10;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
10 rows affected.


[('wiggial01', 512500.0),
 ('lakest01', 60000.0),
 ('landrbi01', 60000.0),
 ('anderla02', 240000.0),
 ('ojedabo01', 650000.0),
 ('washiul01', 80000.0),
 ('conroti01', 115000.0),
 ('marzajo01', 77000.0),
 ('jeffest01', 90000.0),
 ('leachte01', 425000.0)]

In [71]:
%%sql result_4a <<

SELECT playerid, AVG(salaries.salary) FROM salaries  NATURAL JOIN appearances WHERE 
appearances.g_batting = 10 
GROUP BY playerid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
134 rows affected.
Returning data to local variable result_4a


In [72]:
result_4a

[('anderla02', 240000.0),
 ('ashbyan01', 109000.0),
 ('ayraubo01', 125000.0),
 ('bankswi01', 240000.0),
 ('batchri01', 157500.0),
 ('batiski01', 100000.0),
 ('batismi01', 153000.0),
 ('bautijo01', 150000.0),
 ('beller01', 145000.0),
 ('benesan01', 68000.0),
 ('bennega01', 825000.0),
 ('bereja01', 1200000.0),
 ('blackbu02', 3250000.0),
 ('booneaa01', 750000.0),
 ('bouchde01', 135000.0),
 ('bowiemi01', 650000.0),
 ('brantje01', 2800000.0),
 ('burrose01', 525000.0),
 ('buterdr01', 987500.0),
 ('canalge01', 100000.0),
 ('carlybu01', 330000.0),
 ('cartela02', 550000.0),
 ('castiru01', 871429.0),
 ('centeju01', 507500.0),
 ('chacijh01', 4850000.0),
 ('chicoma01', 409000.0),
 ('chrisni01', 500000.0),
 ('colabch01', 521300.0),
 ('conroti01', 115000.0),
 ('cormila01', 405000.0),
 ('cornere01', 109000.0),
 ('crometr01', 109000.0),
 ('damicje01', 2300000.0),
 ('delarjo01', 10000000.0),
 ('dreifda01', 12400000.0),
 ('dunnemi01', 222500.0),
 ('edmonbr01', 170000.0),
 ('eilanda01', 135000.0),
 ('elartsc01', 300000.0),
 ('elsteke01', 300000.0),
 ('erickjo01', 200000.0),
 ('fariepa01', 122500.0),
 ('fasseje01', 750000.0),
 ('fernajo02', 651000.0),
 ('floydcl01', 429500.0),
 ('foxer01', 154000.0),
 ('garcifr02', 10000000.0),
 ('gaudich01', 900000.0),
 ('gedmari01', 920000.0),
 ('greggto01', 375000.0),
 ('grosski01', 165000.0),
 ('hammoje01', 8200000.0),
 ('haneych01', 118000.0),
 ('hershor01', 2000000.0),
 ('higgibo02', 8850000.0),
 ('hillmi01', 158000.0),
 ('hinojda01', 514000.0),
 ('hookch01', 109000.0),
 ('huttoma01', 265000.0),
 ('ilslebl01', 109000.0),
 ('jeffest01', 122500.0),
 ('johnske05', 3000000.0),
 ('johnsmi02', 220000.0),
 ('johnsra05', 9100546.0),
 ('juliojo01', 3600000.0),
 ('jurrjja01', 5500000.0),
 ('kellyro01', 800000.0),
 ('lakest01', 60000.0),
 ('landrbi01', 60000.0),
 ('leachte01', 425000.0),
 ('leiussc01', 275000.0),
 ('lerouch01', 486000.0),
 ('levisje01', 400000.0),
 ('lylesjo01', 2475000.0),
 ('mainejo01', 3300000.0),
 ('martijo07', 400000.0),
 ('martisa01', 318000.0),
 ('marzajo01', 77000.0),
 ('mccarbr01', 12500000.0),
 ('mcglike01', 242000.0),
 ('melusmi01', 200000.0),
 ('mercelu01', 120000.0),
 ('mlickda01', 1250000.0),
 ('moellde01', 112000.0),
 ('molingu01', 380000.0),
 ('molinjo01', 200000.0),
 ('moyerja01', 1100000.0),
 ('natalro01', 147500.0),
 ('nieuwki01', 525415.0),
 ('nieveme01', 118000.0),
 ('nixla01', 390000.0),
 ('nokesma01', 109000.0),
 ('ohkato01', 2750000.0),
 ('ojedabo01', 650000.0),
 ('ottavad01', 1300000.0),
 ('paintla01', 109000.0),
 ('parrila02', 125000.0),
 ('pirklgr01', 112500.0),
 ('pughti01', 182500.0),
 ('ravinjo01', 515000.0),
 ('redmama01', 1000000.0),
 ('remlimi01', 115000.0),
 ('reyesan01', 392500.0),
 ('reyesca01', 500000.0),
 ('riverma01', 10500000.0),
 ('rosenst01', 130000.0),
 ('ruffijo01', 195000.0),
 ('sageraj01', 109000.0),
 ('sanchan01', 390000.0),
 ('schmija01', 2750000.0),
 ('searcst01', 175000.0),
 ('shielja02', 21000000.0),
 ('silvajo01', 650000.0),
 ('slatedo01', 422500.0),
 ('smithgr02', 406000.0),
 ('sotoge01', 3050000.0),
 ('sternad01', 335000.0),
 ('tabakje01', 140000.0),
 ('tomlira01', 975000.0),
 ('tremich01', 109000.0),
 ('tsaoch01', 317000.0),
 ('valdeis01', 5737500.0),
 ('valenjo02', 6350000.0),
 ('varvaan01', 481250.0),
 ('veresra01', 160000.0),
 ('wardtu01', 200000.0),
 ('washiul01', 80000.0),
 ('weberbe01', 600000.0),
 ('wengedo01', 240000.0),
 ('whitema02', 272500.0),
 ('wiggial01', 512500.0),
 ('willida07', 234000.0),
 ('wilsoni01', 125000.0),
 ('woodke02', 3000000.0)]

In [73]:
# Do not delete/edit this cell
result_4a.DataFrame().sort_values('playerid').to_csv('results/result_4a.csv', index=False)

In [74]:
grader.check("q4ai")

q4ai results: All test cases passed!

Inspect the query plan and record the execution time and cost.

In [75]:
result_4a_query_str = "SELECT playerid, AVG(salaries.salary) FROM salaries  NATURAL JOIN appearances WHERE appearances.g_batting = 10 GROUP BY playerid"
!psql -h localhost -d baseball -c "explain analyze $result_4a_query_str"

                                                                                                                     QUERY PLAN                                                                                                                     
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=3637.08..3637.11 rows=1 width=17) (actual time=20.421..20.532 rows=134 loops=1)
   Group Key: salaries.playerid
   ->  Sort  (cost=3637.08..3637.09 rows=1 width=17) (actual time=20.412..20.427 rows=138 loops=1)
         Sort Key: salaries.playerid
         Sort Method: quicksort  Memory: 35kB
         ->  Hash Join  (cost=2900.30..3637.07 rows=1 width=17) (actual time=12.255..20.237 rows=138 loops=1)
               Hash Cond: ((salaries.yearid = appearances.yearid) AND ((salaries.te

In [76]:
result_4a_cost = 3678.06
result_4a_timing = 24.475

In [77]:
grader.check("q4aii")

q4aii results: All test cases passed!

### Question 4b:
Add an index with name `g_batting_idx` on the `g_batting` column of the `appearances` table.

In [78]:
%%sql result_4b <<
CREATE INDEX g_batting_idx
ON appearances (g_batting);

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4b


Now, re-inspect the query plan of the query from `Question 4a` and record its execution time and cost.

In [79]:
!psql -h localhost -d baseball -c "explain analyze $result_4a_query_str"

                                                                                                                     QUERY PLAN                                                                                                                     
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=2381.15..2381.18 rows=1 width=17) (actual time=11.473..11.543 rows=134 loops=1)
   Group Key: salaries.playerid
   ->  Sort  (cost=2381.15..2381.16 rows=1 width=17) (actual time=11.464..11.473 rows=138 loops=1)
         Sort Key: salaries.playerid
         Sort Method: quicksort  Memory: 35kB
         ->  Hash Join  (cost=1644.37..2381.14 rows=1 width=17) (actual time=3.295..11.294 rows=138 loops=1)
               Hash Cond: ((salaries.yearid = appearances.yearid) AND ((salaries.tea

In [80]:
result_4b_cost = 2426.01
result_4b_timing = 17.466

In [81]:
grader.check("q4b")

q4b results: All test cases passed!

In the following question, we will explore adding a different index and evaluating the query from `Question 4a`. To avoid any interference by the `g_batting_idx` index, **drop the index before moving onto the next question.**

In [82]:
%%sql 
DROP INDEX IF EXISTS g_batting_idx;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

### Question 4c:
Write a query to add an index with name `salary_idx` on the `salary` column of the `salaries` table.

In [83]:
%%sql result_4c <<
CREATE INDEX salary_idx
ON salaries (salary);

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4c


Re-inspect the query plan of the query from the `Question 4a` and record its execution time and cost.

In [84]:
!psql -h localhost -d baseball -c "explain analyze $result_4a_query_str"

                                                                                                                     QUERY PLAN                                                                                                                     
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=3637.08..3637.11 rows=1 width=17) (actual time=52.041..52.187 rows=134 loops=1)
   Group Key: salaries.playerid
   ->  Sort  (cost=3637.08..3637.09 rows=1 width=17) (actual time=52.024..52.039 rows=138 loops=1)
         Sort Key: salaries.playerid
         Sort Method: quicksort  Memory: 35kB
         ->  Hash Join  (cost=2900.30..3637.07 rows=1 width=17) (actual time=32.206..51.532 rows=138 loops=1)
               Hash Cond: ((salaries.yearid = appearances.yearid) AND ((salaries.te

In [85]:
result_4c_cost = 3678.06
result_4c_timing = 20.523

In [86]:
grader.check("q4c")

q4c results: All test cases passed!

### Question 4d:
#### Question 4di:
Given your findings from inspecting the query plans with no indexes, an index on `g_batting`, and an index on `salary`, answer the following question. Assign the variable `q4d_part1` to a list of all options which are true for the proposed statement.

1. Consider the following statements:
    1. Adding the `g_batting` index did not have a significant impact on the query execution time and cost.
    1. Adding the `g_batting` index did have a significant impact on the query execution time, but not the cost.
    1. Adding the `g_batting` index did have a significant impact on the query cost, but not the execution time.
    1. Adding the `g_batting` index did have a significant impact on the query cost and execution time.
    1. Adding the `salary` index did not have a significant impact on the query execution time and cost.
    1. Adding the `salary` index did have a significant impact on the query execution time, but not the cost.
    1. Adding the `salary` index did have a significant impact on the query cost, but not the execution time.
    1. Adding the `salary` index did have a significant impact on the query cost and execution time.

**Note:** Your answer should be formatted as follows: `q4d_part1 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q4di
points: 2
-->

In [87]:
q4d_part1 = ["D","E"]

In [88]:
grader.check("q4di")

q4di results: All test cases passed!

<!-- BEGIN QUESTION -->

#### Question 4dii:

Explain your answer based on your knowledge from lectures, and details from inspecting the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q4dii
manual: true
points: 2
-->

I choose D E because index in gbatting index significantly reduce the cost and runtime but index on salary does not.
I think the resason is that we group the salary therefore adding salary index doens't help reduce the cost and time.

Bbatting index reduces the data pages the systme need to scan through, so in the gbatting column, it reduce the data pages and narrow down the range we scan through using a B plus tree when joing two table.

<!-- END QUESTION -->



In the following question, we will further explore the impact of indexes on query performance. To avoid any interference by the `salary_idx` index, please drop the index before moving onto the next question.

In [89]:
%%sql 
DROP INDEX IF EXISTS salary_idx;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

### Question 4e:
#### Question 4ei:
Write a query that finds the `playerid`, `yearid`, and `salary` for each player that had played 10 games and batted in 10 games (the number of games in which a player played can be found in the `g_all` column of the `appearances` table). Your query should join the `salaries` and `appearances` table on all the common columns `yearid`, `teamid`, and `playerid`, so feel free to use a natural join.

In [90]:
%%sql result_4ei <<

SELECT a.playerid, a.salary, a.yearid FROM salaries AS a NATURAL JOIN appearances AS b WHERE 
b.g_batting = 10 AND b.g_all = 10;


 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
120 rows affected.
Returning data to local variable result_4ei


In [91]:
# Do not delete/edit this cell
result_4ei.DataFrame().sort_values(['playerid', 'yearid']).to_csv('results/result_4ei.csv', index=False)

In [92]:
grader.check("q4ei_part1")

q4ei_part1 results: All test cases passed!

Inspect the query plan and record the execution time and cost.

In [93]:
result_4ei_query_str = 'SELECT a.playerid, a.salary, a.yearid FROM salaries AS a NATURAL JOIN appearances AS b WHERE b.g_batting = 10 AND b.g_all = 10'
!psql -h localhost -d baseball -c "explain analyze $result_4ei_query_str"

                                                                            QUERY PLAN                                                                             
-------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Nested Loop  (cost=0.29..3296.09 rows=1 width=21) (actual time=8.106..14.984 rows=120 loops=1)
   ->  Seq Scan on appearances b  (cost=0.00..3133.84 rows=20 width=20) (actual time=0.013..11.656 rows=1289 loops=1)
         Filter: ((g_batting = 10) AND (g_all = 10))
         Rows Removed by Filter: 102967
   ->  Index Scan using salaries_pkey on salaries a  (cost=0.29..8.11 rows=1 width=28) (actual time=0.002..0.002 rows=0 loops=1289)
         Index Cond: ((yearid = b.yearid) AND ((teamid)::text = (b.teamid)::text) AND ((lgid)::text = (b.lgid)::text) AND ((playerid)::text = (b.playerid)::text))
 Planning Time: 1.468 ms
 Execution Time: 15.056 ms
(8 rows)



In [94]:
result_4ei_cost = 3314.40
result_4ei_timing = 15.856

In [95]:
grader.check("q4ei_part2")

q4ei_part2 results: All test cases passed!

In [96]:
%%sql 
drop index g_batting_g_all_idx

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
(psycopg2.errors.UndefinedObject) index "g_batting_g_all_idx" does not exist

[SQL: drop index g_batting_g_all_idx]
(Background on this error at: https://sqlalche.me/e/14/f405)


In [97]:
%sql \di

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
26 rows affected.


[('public', 'allstarfull_pkey', 'index', 'jovyan'),
 ('public', 'appearances_pkey', 'index', 'jovyan'),
 ('public', 'awardsmanagers_pkey', 'index', 'jovyan'),
 ('public', 'awardsplayers_pkey', 'index', 'jovyan'),
 ('public', 'awardssharemanagers_pkey', 'index', 'jovyan'),
 ('public', 'awardsshareplayers_pkey', 'index', 'jovyan'),
 ('public', 'batting_pkey', 'index', 'jovyan'),
 ('public', 'battingpost_pkey', 'index', 'jovyan'),
 ('public', 'collegeplaying_pkey', 'index', 'jovyan'),
 ('public', 'fielding_pkey', 'index', 'jovyan'),
 ('public', 'fieldingof_pkey', 'index', 'jovyan'),
 ('public', 'fieldingpost_pkey', 'index', 'jovyan'),
 ('public', 'halloffame_pkey', 'index', 'jovyan'),
 ('public', 'homegames_pkey', 'index', 'jovyan'),
 ('public', 'managers_pkey', 'index', 'jovyan'),
 ('public', 'managershalf_pkey', 'index', 'jovyan'),
 ('public', 'master_pkey', 'index', 'jovyan'),
 ('public', 'parks_pkey', 'index', 'jovyan'),
 ('public', 'pitching_pkey', 'index', 'jovyan'),
 ('public', 'pitchingpost_pkey', 'index', 'jovyan'),
 ('public', 'salaries_pkey', 'index', 'jovyan'),
 ('public', 'schools_pkey', 'index', 'jovyan'),
 ('public', 'seriespost_pkey', 'index', 'jovyan'),
 ('public', 'teams_pkey', 'index', 'jovyan'),
 ('public', 'teamsfranchises_pkey', 'index', 'jovyan'),
 ('public', 'teamshalf_pkey', 'index', 'jovyan')]

#### Question 4eii:
Now, let's see the impact of adding an index on the `g_batting` column. Create an index on the `g_batting` column. Re-inspect the query from `Question 4ei` and record the execution time and cost.

In [98]:
%%sql result_4eii << 
CREATE INDEX g_batting_idx
ON appearances (g_batting);

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4eii


In [99]:
!psql -h localhost -d baseball -c "explain analyze $result_4ei_query_str"

                                                                            QUERY PLAN                                                                             
-------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Nested Loop  (cost=18.75..1782.57 rows=1 width=21) (actual time=4.372..8.649 rows=120 loops=1)
   ->  Bitmap Heap Scan on appearances b  (cost=18.46..1620.32 rows=20 width=20) (actual time=0.419..3.411 rows=1289 loops=1)
         Recheck Cond: (g_batting = 10)
         Filter: (g_all = 10)
         Rows Removed by Filter: 58
         Heap Blocks: exact=899
         ->  Bitmap Index Scan on g_batting_idx  (cost=0.00..18.46 rows=1355 width=0) (actual time=0.260..0.261 rows=1347 loops=1)
               Index Cond: (g_batting = 10)
   ->  Index Scan using salaries_pkey on salaries a  (cost=0.29..8.11 rows=1 width=28) (actual time=0.003..0.003 rows=0 loops=1289)
     

In [100]:
result_4eii_with_index_cost = 1804.82
result_4eii_with_index_timing = 6.090

In [101]:
grader.check("q4eii")

q4eii results: All test cases passed!

In the following question, we will further explore the impact of indexes on query performance. To avoid any interference from the index on the `g_batting` column, drop the index before moving onto the next question.

In [102]:
%%sql 
DROP INDEX IF EXISTS g_batting_idx;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

#### Question 4eiii:
Write a query that finds the `playerid`, `yearid`, and `salary` for each player that had played 10 games __or__ batted in 10 games. Your query should join the `salaries` and `appearances` table on all the common columns `yearid`, `teamid`, and `playerid`, so feel free to use a natural join.

In [103]:
%%sql result_4eiii <<
SELECT a.playerid, a.salary, a.yearid FROM salaries AS a NATURAL JOIN appearances AS b WHERE 
b.g_batting = 10 OR b.g_all = 10;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
216 rows affected.
Returning data to local variable result_4eiii


In [104]:
# Do not delete/edit this cell
result_4eiii.DataFrame().sort_values(['playerid', 'yearid']).to_csv('results/result_4eiii.csv', index=False)

In [105]:
grader.check("q4eiii_part1")

q4eiii_part1 results: All test cases passed!

In [106]:
result_4eiii_query_str = "SELECT a.playerid, a.salary, a.yearid FROM salaries AS a NATURAL JOIN appearances AS b WHERE b.g_batting = 10 OR b.g_all = 10"
!psql -h localhost -d baseball -c "explain analyze $result_4eiii_query_str"

                                                                             QUERY PLAN                                                                             
--------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=3191.48..3928.25 rows=1 width=21) (actual time=13.991..21.901 rows=216 loops=1)
   Hash Cond: ((a.yearid = b.yearid) AND ((a.teamid)::text = (b.teamid)::text) AND ((a.lgid)::text = (b.lgid)::text) AND ((a.playerid)::text = (b.playerid)::text))
   ->  Seq Scan on salaries a  (cost=0.00..459.28 rows=26428 width=28) (actual time=0.016..2.016 rows=26428 loops=1)
   ->  Hash  (cost=3133.84..3133.84 rows=2882 width=20) (actual time=13.788..13.790 rows=1655 loops=1)
         Buckets: 4096  Batches: 1  Memory Usage: 118kB
         ->  Seq Scan on appearances b  (cost=0.00..3133.84 rows=2882 width=20) (actual time=0.012..13.125 rows=1655 loops=1)
      

In [107]:
result_4eiii_cost = 3970.35
result_4eiii_timing = 22.454

In [108]:
grader.check("q4eiii_part2")

q4eiii_part2 results: All test cases passed!

#### Question 4eiv
Now, let's see the impact of adding an index on `g_batting` column will have on the query. Re-create the index and re-inspect the query from `Question 4eiii` and record the execution time and cost.

In [109]:
%%sql result_4eiv << 
CREATE INDEX g_batting_idx
ON appearances (g_batting);

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4eiv


In [110]:
!psql -h localhost -d baseball -c "explain analyze $result_4eiii_query_str"

                                                                             QUERY PLAN                                                                             
--------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=3191.48..3928.25 rows=1 width=21) (actual time=21.211..28.917 rows=216 loops=1)
   Hash Cond: ((a.yearid = b.yearid) AND ((a.teamid)::text = (b.teamid)::text) AND ((a.lgid)::text = (b.lgid)::text) AND ((a.playerid)::text = (b.playerid)::text))
   ->  Seq Scan on salaries a  (cost=0.00..459.28 rows=26428 width=28) (actual time=0.024..1.908 rows=26428 loops=1)
   ->  Hash  (cost=3133.84..3133.84 rows=2882 width=20) (actual time=20.982..20.983 rows=1655 loops=1)
         Buckets: 4096  Batches: 1  Memory Usage: 118kB
         ->  Seq Scan on appearances b  (cost=0.00..3133.84 rows=2882 width=20) (actual time=0.017..20.042 rows=1655 loops=1)
      

In [111]:
result_4eiv_with_index_cost = 3970.35
result_4eiv_with_index_timing = 21.707

In [112]:
grader.check("q4eiv")

q4eiv results: All test cases passed!

In the following question, we will further explore the impact of indexes on query performance. To avoid any interference from the index on the `g_batting` column, drop the index before moving onto the next question.

In [113]:
%%sql 
DROP INDEX IF EXISTS g_batting_idx;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

#### Question 4ev:
Now, create a multiple column index on `g_batting` and `g_all` called `g_batting_g_all_idx` and record the query execution time and cost.

In [114]:
%%sql result_4v <<
create index g_batting_g_all_idx on appearances(g_batting,g_all);

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4v


In [115]:
!psql -h localhost -d baseball -c "explain analyze $result_4eiii_query_str"

                                                                             QUERY PLAN                                                                             
--------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=2881.09..3617.87 rows=1 width=21) (actual time=4.286..12.389 rows=216 loops=1)
   Hash Cond: ((a.yearid = b.yearid) AND ((a.teamid)::text = (b.teamid)::text) AND ((a.lgid)::text = (b.lgid)::text) AND ((a.playerid)::text = (b.playerid)::text))
   ->  Seq Scan on salaries a  (cost=0.00..459.28 rows=26428 width=28) (actual time=0.014..1.976 rows=26428 loops=1)
   ->  Hash  (cost=2823.45..2823.45 rows=2882 width=20) (actual time=4.073..4.076 rows=1655 loops=1)
         Buckets: 4096  Batches: 1  Memory Usage: 118kB
         ->  Bitmap Heap Scan on appearances b  (cost=1182.11..2823.45 rows=2882 width=20) (actual time=1.418..3.598 rows=1655 loops=1)

In [116]:
result_4ev_multiple_col_index_cost = 3655.54
result_4ev_multiple_col_index_timing = 19.354

In [117]:
grader.check("q4ev")

q4ev results: All test cases passed!

#### Question 4evi:
Given your findings from inspecting the query plans from `Question 4e`, answer the following question. Assign the variable `q4e_part6` to a list of all statements that are true.

1. Consider the following statements:
    1. Adding an index on a column used in an AND predicate will reduce the query time, but not the execution cost.
    1. Adding an index on a column used in an AND predicate will reduce the query cost, but not the execution time.
    1. Adding an index on a column used in an AND predicate will reduce the query cost and the execution time.
    1. Adding an index on a column used in an OR predicate will reduce the query time, but not the execution cost.
    1. Adding an index on a column used in an OR predicate will reduce the query cost, but not the execution time.
    1. Adding an index on a column used in an OR predicate will reduce the query cost and the execution time.
    1. Adding a mutlicolumn index on columns in an OR predicate will reduce the query time, but not the execution cost.
    1. Adding a mutlicolumn index on columns in an OR predicate will reduce the query cost, but not the execution time.
    1. Adding a mutlicolumn index on columns in an OR predicate will reduce the query cost and the execution time.

**Note:** Your answer should be formatted as follows: `q4e_part6 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q4evi
points: 2
-->

In [118]:
q4e_part6 = ["C","I"]

In [119]:
grader.check("q4evi")

q4evi results: All test cases passed!

<!-- BEGIN QUESTION -->

#### Question 4evii

Explain your answer to the previous part based on your knowledge from lectures, and details of the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q4evii
manual: true
points: 2
-->

I choose "C I" because adding gbating index reduce cost and time for the AND case, and adding multicolumn index for OR case reduce cost and time.

For C because the system perform a nested loop join, addding a index on a column reduce the data pages the systme need to scan through when joining two table.

For "I" we are not joing to table becase the condition is OR. So we need to look for both columns in the two table and a multicolumn index help narrorw down range when we searching for value.

In [120]:
%%sql 
drop index g_batting_g_all_idx

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

<!-- END QUESTION -->



## Question 5: More Indexes
### Question 5a:
Write 2 queries: one that finds the minimum salary from the salary table and one that finds the average. Inspect the queries' query plans and record their execution times and costs.

In [121]:
%%sql result_5a_min << 
SELECT MIN(salary) FROM salaries;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
1 rows affected.
Returning data to local variable result_5a_min


In [122]:
result_5a_min_query_str = "SELECT MIN(salary) FROM salaries"
!psql -h localhost -d baseball -c "explain analyze $result_5a_min_query_str"

                                                    QUERY PLAN                                                    
------------------------------------------------------------------------------------------------------------------
 Aggregate  (cost=525.35..525.36 rows=1 width=8) (actual time=4.409..4.410 rows=1 loops=1)
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=8) (actual time=0.015..1.976 rows=26428 loops=1)
 Planning Time: 0.400 ms
 Execution Time: 4.502 ms
(4 rows)



In [123]:
result_5a_min_query_cost = 544.22
result_5a_min_query_timing = 4.205

In [124]:
grader.check("q5ai")

q5ai results: All test cases passed!

In [125]:
%%sql result_5a_avg <<
SELECT AVG(salary) FROM salaries

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
1 rows affected.
Returning data to local variable result_5a_avg


In [126]:
result_5a_avg_query_str = 'SELECT AVG(salary) FROM salaries'
!psql -h localhost -d baseball -c "explain analyze $result_5a_avg_query_str"

                                                    QUERY PLAN                                                    
------------------------------------------------------------------------------------------------------------------
 Aggregate  (cost=525.35..525.36 rows=1 width=8) (actual time=4.736..4.738 rows=1 loops=1)
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=8) (actual time=0.014..1.883 rows=26428 loops=1)
 Planning Time: 0.311 ms
 Execution Time: 4.798 ms
(4 rows)



In [127]:
result_5a_avg_query_cost = 544.23
result_5a_avg_query_timing = 4.813

In [128]:
grader.check("q5aii")

q5aii results: All test cases passed!

### Question 5b:
Create an index on the `salary` column in the `salaries` table and re-inspect the query plans from the previous part and record the respective execution time and cost.

In [129]:
%%sql 
create index index_salary on salaries(salary);

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

In [130]:
!psql -h localhost -d baseball -c "explain analyze $result_5a_min_query_str"

                                                                   QUERY PLAN                                                                   
------------------------------------------------------------------------------------------------------------------------------------------------
 Result  (cost=0.32..0.33 rows=1 width=8) (actual time=0.059..0.059 rows=1 loops=1)
   InitPlan 1 (returns $0)
     ->  Limit  (cost=0.29..0.32 rows=1 width=8) (actual time=0.056..0.057 rows=1 loops=1)
           ->  Index Only Scan using index_salary on salaries  (cost=0.29..762.78 rows=26428 width=8) (actual time=0.054..0.055 rows=1 loops=1)
                 Index Cond: (salary IS NOT NULL)
                 Heap Fetches: 0
 Planning Time: 0.567 ms
 Execution Time: 0.125 ms
(8 rows)



In [131]:
result_5b_min_query_cost = 0.33
result_5b_min_query_timing = 0.164

In [132]:
grader.check("q5bi")

q5bi results: All test cases passed!

In [133]:
!psql -h localhost -d baseball -c "explain analyze $result_5a_avg_query_str"

                                                    QUERY PLAN                                                    
------------------------------------------------------------------------------------------------------------------
 Aggregate  (cost=525.35..525.36 rows=1 width=8) (actual time=4.685..4.686 rows=1 loops=1)
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=8) (actual time=0.018..1.905 rows=26428 loops=1)
 Planning Time: 0.500 ms
 Execution Time: 4.783 ms
(4 rows)



In [134]:
result_5b_avg_query_cost = 532.36
result_5b_avg_query_timing = 7.056

In [135]:
grader.check("q5bii")

q5bii results: All test cases passed!

In the following questions, we will further explore the impact of indexes on query performance. To avoid any interference from the index on the `salary` column, drop the index before moving onto the next question.

In [136]:
%%sql 
drop index index_salary

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

### Question 5c:
Given your findings from `Question 5a-b`, answer the following question. Assign the variable `q5c` to the correct answer choice.

1. Which of the following statements is true?
    1. An index on the column being aggregated in a query will always provide a performance enhancement.
    1. A query finding the MIN(salary) will always benefit from an index on salary, but a query finding MAX(salary) will not.
    1. A query finding the COUNT(salary) will always benefit from an index on salary, but a query finding AVG(salary) will not.
    1. Queries finding the MIN(salary) or MAX(salary) will always benefit from an index on salary, but queries finding AVG(salary) or COUNT(salary) will not.

*Note:* Your answer should be formatted as follows: `q5c = ['A', 'B']`

<!--
BEGIN QUESTION
name: q5c
points: 1
-->

In [137]:
q5c = ["D"]

In [138]:
grader.check("q5c")

q5c results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 5d:

Explain your answer to the previous part based on your knowledge from lectures, and your inspection of the query plans.

<!--
BEGIN QUESTION
name: q5d
manual: true
points: 2
-->

When adding a index to find min the cost and time siginificantly reduce so I choose D.

When we need to find a MIN or MAX without index, we need to scan through every number in the column.
Adding an index to the column can reduce the pages systme need to scan thorugh and narrow down the range of search,
and thus reduce the cost and time.

<!-- END QUESTION -->



## Question 6: Clustered Indexes
In this question, we will inspect the impact that clustering our data on an index can have on a query's performance.

### Question 6a
Write a query that finds the `playerid`, `yearid`, `teamid`, and `ab` for all players whose `ab` was above 500. Inspect the query plan and record the execution time and cost.

In [139]:
%%sql result_6a <<
SELECT playerid, yearid, teamid, ab FROM batting WHERE ab > 500;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
8839 rows affected.
Returning data to local variable result_6a


In [140]:
# Do not delete/edit this cell
result_6a.DataFrame().sort_values(['playerid', 'yearid', 'teamid']).iloc[:1000].to_csv('results/result_6a.csv', index=False)

In [141]:
grader.check("q6ai")

q6ai results: All test cases passed!

In [142]:
result_6a_query_str = "SELECT playerid, yearid, teamid, ab FROM batting WHERE ab > 500"
!psql -h localhost -d baseball -c "explain analyze $result_6a_query_str"

                                                 QUERY PLAN                                                 
------------------------------------------------------------------------------------------------------------
 Seq Scan on batting  (cost=0.00..2884.05 rows=8694 width=21) (actual time=0.419..14.185 rows=8839 loops=1)
   Filter: (ab > 500)
   Rows Removed by Filter: 95485
 Planning Time: 0.513 ms
 Execution Time: 14.580 ms
(5 rows)



In [160]:
result_6a_cost = 2884.05
result_6a_timing = 14.580

In [161]:
grader.check("q6aii")

q6aii results: All test cases passed!

### Question 6b:
Cluster the `batting` table on its primary key (hint: use `\di` to find out what name of the primary key is). We are able to directly cluster on the primary key (without first creating a separate index) because Postgres automatically creates an index for it. Re-inspect the query plan for the query from `Question 6a` and record the execution time and cost.

In [145]:
%sql \di

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
26 rows affected.


[('public', 'allstarfull_pkey', 'index', 'jovyan'),
 ('public', 'appearances_pkey', 'index', 'jovyan'),
 ('public', 'awardsmanagers_pkey', 'index', 'jovyan'),
 ('public', 'awardsplayers_pkey', 'index', 'jovyan'),
 ('public', 'awardssharemanagers_pkey', 'index', 'jovyan'),
 ('public', 'awardsshareplayers_pkey', 'index', 'jovyan'),
 ('public', 'batting_pkey', 'index', 'jovyan'),
 ('public', 'battingpost_pkey', 'index', 'jovyan'),
 ('public', 'collegeplaying_pkey', 'index', 'jovyan'),
 ('public', 'fielding_pkey', 'index', 'jovyan'),
 ('public', 'fieldingof_pkey', 'index', 'jovyan'),
 ('public', 'fieldingpost_pkey', 'index', 'jovyan'),
 ('public', 'halloffame_pkey', 'index', 'jovyan'),
 ('public', 'homegames_pkey', 'index', 'jovyan'),
 ('public', 'managers_pkey', 'index', 'jovyan'),
 ('public', 'managershalf_pkey', 'index', 'jovyan'),
 ('public', 'master_pkey', 'index', 'jovyan'),
 ('public', 'parks_pkey', 'index', 'jovyan'),
 ('public', 'pitching_pkey', 'index', 'jovyan'),
 ('public', 'pitchingpost_pkey', 'index', 'jovyan'),
 ('public', 'salaries_pkey', 'index', 'jovyan'),
 ('public', 'schools_pkey', 'index', 'jovyan'),
 ('public', 'seriespost_pkey', 'index', 'jovyan'),
 ('public', 'teams_pkey', 'index', 'jovyan'),
 ('public', 'teamsfranchises_pkey', 'index', 'jovyan'),
 ('public', 'teamshalf_pkey', 'index', 'jovyan')]

In [148]:
%%sql 
CLUSTER batting using batting_pkey;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

In [149]:
!psql -h localhost -d baseball -c "explain analyze $result_6a_query_str"

                                                 QUERY PLAN                                                 
------------------------------------------------------------------------------------------------------------
 Seq Scan on batting  (cost=0.00..2878.05 rows=8694 width=21) (actual time=0.010..16.081 rows=8839 loops=1)
   Filter: (ab > 500)
   Rows Removed by Filter: 95485
 Planning Time: 0.420 ms
 Execution Time: 16.466 ms
(5 rows)



In [150]:
result_6b_cost = 2878.05
result_6b_timing = 16.466

In [151]:
grader.check("q6b")

q6b results: All test cases passed!

### Question 6c
Now, let's try clustering the table based on another index. Create an index on `ab` column called `ab_idx` in the `batting` table AND cluster the `batting` table with this new index. Now, re-inspect the query plan and record the execution time and cost.

In [153]:
%%sql 
create index ab_idx  on batting(ab);


 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
(psycopg2.errors.DuplicateTable) relation "ab_idx" already exists

[SQL: create index ab_idx on batting(ab);]
(Background on this error at: https://sqlalche.me/e/14/f405)


In [154]:
%%sql 
CLUSTER batting using ab_idx

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

In [155]:
!psql -h localhost -d baseball -c "explain analyze $result_6a_query_str"

                                                      QUERY PLAN                                                      
----------------------------------------------------------------------------------------------------------------------
 Bitmap Heap Scan on batting  (cost=99.67..1783.35 rows=8694 width=21) (actual time=0.486..3.397 rows=8839 loops=1)
   Recheck Cond: (ab > 500)
   Heap Blocks: exact=135
   ->  Bitmap Index Scan on ab_idx  (cost=0.00..97.50 rows=8694 width=0) (actual time=0.448..0.448 rows=8839 loops=1)
         Index Cond: (ab > 500)
 Planning Time: 0.627 ms
 Execution Time: 3.958 ms
(7 rows)



In [156]:
result_6c_cost = 1783.35
result_6c_timing = 3.958

In [157]:
grader.check("q6c")

q6c results: All test cases passed!

### Question 6d
Given your findings from inspecting the query plans from `Question 6a-c`, answer the following question. Assign the variable `q6d` to a list of all statements that are true.

1. Consider the following statements:
    1. Clustering based on the `ab_idx` decreased the cost of the query.
    1. Clustering based on the `ab_idx` increased the cost of the query.
    1. Clustering based on the `ab_idx` increased the execution time of the query.
    1. Clustering based on the `ab_idx` decreased the execution time of the query.
    1. Clustering based on the `batting_pkey` decreased the cost of the query.
    1. Clustering based on the `batting_pkey` increased the cost of the query.
    1. Clustering based on the `batting_pkey` increased the execution time of the query.
    1. Clustering based on the `batting_pkey` decreased the execution time of the query.
    1. None of the above
    
**Note:** Your answer should be formatted as follows: `q6d = ['A', 'B']`

<!--
BEGIN QUESTION
name: q6d
points: 2
-->

In [162]:
q6d =["A","D"]

In [163]:
grader.check("q6d")

q6d results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 6e:

Explain your answer to the previous part based on your knowledge from lectures, and your inspection of the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q6e
manual: true
points: 2
-->

 I choose "A", "D" because cluster on index reduce time and cost.

Clustering on the primary key will not always reduce cost and execution time because if the query is filtered on another column that's  primary key, the execution time could stay the same or worsen due to traversing the index.
 
Clustering on index means that the rows in the data pages are in the same order as the rows in the index. It reduces the data pages the system need to scan through and narrow down the rage due to the indexes, and thus cost less and spend less time. For exmaple, if we knew that all values of A = a are together (as opposed to randomly shuffled), we can simply read one block as opposed to 100 blocks
 

<!-- END QUESTION -->



## Question 7: Cost of Index Management
Until now, we have seen the positive potential impact that indexes can have on query performance, but remember in real world technologies/applications, we will be routinely receiving new data (and in large quantities) which would trigger regular updates to our tables. In this section, we will dive into the cost of managing the indexes that we create.

### Question 7a:
Record the time it takes to insert 300,000 rows into the `salaries` table when no additional index is configured.

Run the following cell to setup a column to track which rows we added as part of these inserts.

In [164]:
%sql ALTER TABLE salaries ADD added boolean DEFAULT False;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

Next, run the provided update script and record the **wall time**.

**NOTE:** Running the below cell multiple times may result in an error, unless you first delete the rows with the cell given at the end of this subpart.

In [165]:
%%time
%%sql
DO $$
 DECLARE counter INTEGER := 1;
 BEGIN
     FOR counter IN 100001..400000 LOOP
     INSERT INTO salaries (yearid, teamid, lgid, playerid, salary, added)
         VALUES (2021, 'ATL', 'NL', 'p' || counter, RANDOM() * 1000000, true);
     END LOOP;
END;
$$;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
CPU times: user 6.02 ms, sys: 1.36 ms, total: 7.38 ms
Wall time: 3.73 s


[]

In [167]:
result_7a_timing = 3.73 

In [168]:
grader.check("q7a")

q7a results: All test cases passed!

Before adding an index to the salaries table and re-timing the updates, delete all the rows that were added to the table from the update script.

In [169]:
%%sql
DELETE FROM salaries
WHERE added = 'true';

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
300000 rows affected.


[]

### Question 7b:
Now, create an index on the `salary` column and record the **wall time** after executing the update script.

In [170]:
%%sql 
CREATE INDEX salary_idx
ON salaries (salary);

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

**NOTE:** Running the below cell multiple times may result in an error, unless you first delete the rows with the cell given at the end of last subpart.

In [171]:
%%time
%%sql
DO $$
 DECLARE counter INTEGER := 1;
 BEGIN
     FOR counter IN 100001..400000 LOOP
     INSERT INTO salaries (yearid, teamid, lgid, playerid, salary, added)
         VALUES (2021, 'ATL', 'NL', 'p' || counter, RANDOM() * 1000000, true);
     END LOOP;
END;
$$;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
CPU times: user 4.79 ms, sys: 344 µs, total: 5.14 ms
Wall time: 5.07 s


[]

In [174]:
result_7b_timing =5.07 

In [175]:
grader.check("q7b")

q7b results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 7c:
What difference did you notice when you added an index into the salaries table and re-timed the update? Why do you think it happened?

<!--
BEGIN QUESTION
name: q7c
manual: true
points: 2
-->

It took more time when adding the index because index is not very good on tables that are modified more often than queried. If an index is not maintained, it may lead to inaccurate query results, and therefore if a new tuple is inserted but it is not reflected in the index, it won’t be returned as part of queries and it cost more to scan.
In addition, a constennly updating table may have low cardinality which index may not perform well. 

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

## Question 8: Takeaway

In this project, we explored how the database system optimizes query execution and how users can futher tune the performance of their queries.

Familiarizing yourself with these optimization and tuning methods will make you a better data engineer. In this question, we'll ask you to recall and summarize these concepts. Who knows? Maybe one day it will help you during an interview or on a project.

In the following answer cell,
1. Name 3 methods you learned in this project. The method can be either the optimization done by the database system, or the fine tuning done by the user.
2. For each method, summarize how and why it can optimize query performance. Feel free to discuss any drawbacks, if applicable.

<!--
BEGIN QUESTION
name: q8
manual: true
points: 6
-->

Method 1: Create a table with materialized views cost less and spend less time because since materialized views are stored as "tables" in the system and there is no need to re-compute.
However, there is a higher matainance cost to it.
In contarst, Views are computed each time that you need to use the view which cost more.

Method 2: If tables are in sorted order, using a Merge Join, which sorted two tables first and then merge and match tuples across runs by walking down the runs in sorted order, could be more effcienct. The drawback is the table must be in sorted order in the first place in order to reduce cost.

Method3: When joining 2 tables, we can index one column because the system perform a nested loop join, addding a index on a column reduce the data pages the systme need to scan through when joining two table.


<!-- END QUESTION -->



## Congratulations! You have finished Project 2.

Run the following cell to zip and download the results of your queries. You will also need to run the export cell at the end of the notebook.

**Please save your notebook before exporting (this is a good time to do it!)** Otherwise, we may not be able to register your written responses.

**For submission on Gradescope, you will need to submit BOTH the proj2.zip file genreated by the export cell and the results.zip file generated by the following cell.**

**Common submission issues:** You MUST submit the generated zip files (not folders) to the autograder. However, Safari is known to automatically unzip files upon downloading. You can fix this by going into Safari preferences, and deselect the box with the text "Open safe files after downloading" under the "General" tab. If you experience issues with downloading via clicking on the link, you can also navigate to the project 2 directory within JupyterHub (remove `proj2.ipynb` from the url), and manually download the generated zip files. Please post on Ed if you encounter any other submission issues.

In [199]:
from IPython.display import display, FileLink

!zip -r results.zip results
results_file = FileLink('./results.zip', result_html_prefix="Click here to download: ")
display(results_file)

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export()